In [66]:
import pickle

import psycopg2

import pandas as pd
import pandas.io.sql as sqlio

from gensim.similarities.docsim import Similarity
from gensim.corpora import MmCorpus
from gensim.matutils import corpus2csc

from joblib import Parallel, delayed, cpu_count
import funcy as fp

In [103]:
def save_pickle(file_name, object_to_pickle):
    with open(file_name, 'wb') as f:
        pickle.dump(object_to_pickle, f)
        
def load_pickle(file_name):
    with open(file_name, 'rb') as f:
        unpickled_object = pickle.load(f)
    return unpickled_object

In [62]:
def _series_chunks(s, n_jobs):
    if n_jobs < 0:
        # so, have n chunks if we are using all n cores/cpus = cpu_count() + 1 + n_jobs
        n_jobs = cpu_count() + 1 + n_jobs
    n = len(s)
    n_chunks = int(n / n_jobs)
    return (s.iloc[ilocs] for ilocs in fp.chunks(n_chunks, range(n)))

def series_pmap(s, f, n_jobs=-1):
    if n_jobs == 0:
        return s.map(f)
    return pd.concat(Parallel(n_jobs=n_jobs)(delayed(series_pmap)(sub_series, f, n_jobs=0) \
                                                 for sub_series in _series_chunks(s, n_jobs)))

In [28]:
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres")
stories_df = sqlio.read_sql_query("SELECT id, score, author, title, url FROM bq_all WHERE all_text IS NOT NULL AND all_text != '' AND article_content IS NOT NULL AND article_content != '' ORDER BY score DESC, story_time DESC LIMIT 25000", conn, index_col='id')

CPU times: user 43.3 ms, sys: 15.6 ms, total: 58.9 ms
Wall time: 615 ms


In [83]:
tag_names = ['Python',
 'Mobile',
 'Design',
 'Security',
 'Blockchain',
 'AI/Machine Learning',
 'Google',
 'Microsoft',
 'Apple',
 'Facebook',
 'Amazon',
 'Startups',
 'Politics',
 'Databases',
 'Linux',
 'Data Science',
 'Science',
 'Math',
 'Javascript',
 'Web Dev',
 'DevOps',
 'Hardware/IoT',
 'AR/VR',
 'Games']

id2index_map = {story_id: index for index, story_id in enumerate(stories_df.index)}

In [99]:
def get_series_index(story_id):
    return id2index_map[story_id]

def get_sim_content_ids(story_id):
    series_index = get_series_index(story_id)
    vec = content_corpus[series_index]
    sims = content_indexer[vec]
    sim_stories = [stories_df.index[sim_index] for sim_index, sim_score in sims]
    return sim_stories[1:]

def get_sim_user_ids(story_id):
    series_index = get_series_index(story_id)
    vec = user_corpus[series_index]
    sims = user_indexer[vec]
    sim_stories = [stories_df.index[sim_index] for sim_index, sim_score in sims]
    return sim_stories[1:]

In [7]:
content_indexer = Similarity.load('lsi_300_indexer.model')
content_corpus = MmCorpus('lsi_300_corpus.mm')
content_csc = corpus2csc(content_corpus)

In [98]:
user_indexer = Similarity.load('commenters_indexer.model')
user_corpus = MmCorpus('commenters_corpus.mm')

In [71]:
tagger_model = load_pickle('tagger_model.pkl')
tag_results = tagger_model.predict(content_csc.T).todense()

In [75]:
tagged_stories_df = pd.DataFrame(tag_results, columns=tag_names)
tagged_stories_df["story_id"] = stories_df.index.values
tagged_stories_df = tagged_stories_df.set_index(keys=["story_id"], drop=False)
stories_df[tag_names] = tagged_stories_df[tag_names]

In [76]:
stories_df.head()

,score,author,title,url,Python,Mobile,Design,Security,Blockchain,AI/Machine Learning,...,Data Science,Science,Math,Javascript,Web Dev,DevOps,Hardware/IoT,AR/VR,Games,story_id
id,,,,,,,,,,,,,,,,,,,,,
16582136,6015,Cogito,Stephen Hawking has died,http://www.bbc.com/news/uk-43396008,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,16582136
11116274,5771,epaga,A Message to Our Customers,http://www.apple.com/customer-letter/,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11116274
13682022,4107,grey-area,"Reflecting on one very, very strange year at Uber",https://www.susanjfowler.com/blog/2017/2/19/re...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,13682022
13718752,3238,tptacek,Cloudflare Reverse Proxies Are Dumping Uniniti...,https://bugs.chromium.org/p/project-zero/issue...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,13718752
11966167,3125,dmmalam,UK votes to leave EU,http://www.bbc.co.uk/news/uk-politics-36615028,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11966167


In [92]:
%%time 

stories_df["content_recs"] = series_pmap(stories_df["story_id"], get_sim_content_ids)

CPU times: user 485 ms, sys: 152 ms, total: 636 ms
Wall time: 8min 40s


In [97]:
stories_df["content_recs"].head()

id
16582136    [12238197, 16583822, 12062988, 10768391, 12994...
11116274    [11150107, 11116801, 11236416, 11133082, 11155...
13682022    [13747414, 13684439, 15524669, 13725529, 13784...
13718752    [10192273, 13766339, 12105188, 10257932, 12091...
11966167    [13985192, 11964880, 13856961, 11967478, 15568...
Name: content_recs, dtype: object

In [100]:
%%time 

stories_df["user_recs"] = series_pmap(stories_df["story_id"], get_sim_user_ids)

CPU times: user 574 ms, sys: 168 ms, total: 742 ms
Wall time: 9min 31s


In [101]:
stories_df["user_recs"].head()

id
16582136      [11771737, 4372563, 8504931, 14112748, 3719005]
11116274    [12772925, 15253659, 16476454, 12456569, 12180...
13682022      [4755470, 13902938, 7250505, 11377716, 8598652]
13718752    [16077873, 15948489, 6991590, 16216329, 15843957]
11966167     [15178970, 12657249, 6312100, 6859245, 14958329]
Name: user_recs, dtype: object

In [104]:
save_pickle('stories_df.pkl', stories_df)

In [105]:
stories_df.head()

,score,author,title,url,Python,Mobile,Design,Security,Blockchain,AI/Machine Learning,...,Math,Javascript,Web Dev,DevOps,Hardware/IoT,AR/VR,Games,story_id,content_recs,user_recs
id,,,,,,,,,,,,,,,,,,,,,
16582136,6015,Cogito,Stephen Hawking has died,http://www.bbc.com/news/uk-43396008,0,0,0,0,0,0,...,0,0,0,0,0,0,0,16582136,"[12238197, 16583822, 12062988, 10768391, 12994...","[11771737, 4372563, 8504931, 14112748, 3719005]"
11116274,5771,epaga,A Message to Our Customers,http://www.apple.com/customer-letter/,0,1,0,0,0,0,...,0,0,0,0,0,0,0,11116274,"[11150107, 11116801, 11236416, 11133082, 11155...","[12772925, 15253659, 16476454, 12456569, 12180..."
13682022,4107,grey-area,"Reflecting on one very, very strange year at Uber",https://www.susanjfowler.com/blog/2017/2/19/re...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,13682022,"[13747414, 13684439, 15524669, 13725529, 13784...","[4755470, 13902938, 7250505, 11377716, 8598652]"
13718752,3238,tptacek,Cloudflare Reverse Proxies Are Dumping Uniniti...,https://bugs.chromium.org/p/project-zero/issue...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,13718752,"[10192273, 13766339, 12105188, 10257932, 12091...","[16077873, 15948489, 6991590, 16216329, 15843957]"
11966167,3125,dmmalam,UK votes to leave EU,http://www.bbc.co.uk/news/uk-politics-36615028,0,0,0,0,0,0,...,0,0,0,0,0,0,0,11966167,"[13985192, 11964880, 13856961, 11967478, 15568...","[15178970, 12657249, 6312100, 6859245, 14958329]"


In [106]:
summaries_df = sqlio.read_sql_query("SELECT id, article_summary FROM bq_all WHERE all_text IS NOT NULL AND all_text != '' AND article_content IS NOT NULL AND article_content != '' ORDER BY score DESC, story_time DESC LIMIT 25000", conn, index_col='id')

In [126]:
save_pickle('summaries_df.pkl', summaries_df)

In [112]:
pd.set_option('display.max_colwidth', 500)

In [ ]:
Media playback is unsupported on your device Media caption

In [125]:
summaries_df.sample(500)

,article_summary
id,
12573173,"Society has a rich history of people seizing on social evolution as an excuse for bad manners.\nGood manners are mere mannerisms, the argument goes, which serve only to put barriers in the way of deeper connections.\nIt’s the looser ties, the ones that have to be created or re-created at each meeting, that are tough.\nWhen I skip big gatherings of strangers, I’m not just being a little rude to the individual people around me, I’m being uncivil in a larger sense.\nThat doesn’t have to mean we..."
8644175,"Their team is growing because of advancements they've made, and you want to recognize the work they've done with something.\nThe obvious answer is to put them in charge of the team they've built, especially as they're the de-facto leader of the team already.\nThose are the fun parts of any engineer's job, and indeed all engineers must exercise some degree of technical leadership.\nDeveloping a strategy for positioning your technical thought leaders as the technical thought leaders is empower..."
4095814,"The MacBook Pro got a widely anticipated Retina Display graphics update today, along with a new design that makes it almost as lightweight as a MacBook Air.\nNo updates to any 17-inch MacBook Pro models were announced.\nThe premium 15-inch model is even thinner than the 13-inch MacBook Pro, which comes in at 0.95 inches thick and 4.5 pounds.\nThe ""low-end"" 15-inch MacBook Pro is also 0.95 inches thick, weighing 5.6 pounds, with prices starting at $1,799.\nWhile the 13-inch MacBook Pro will o..."
16258263,"Riot.im is the powerful decentralized collaboration tool built on the Matrix protocol.\n“It gives us the freedom to expand our team, and continue improving Riot.im and Matrix in an open source environment.\nBuilt on the Matrix open protocol, Riot.im bridges users to any Matrix-bridged apps (such as IRC and Slack) and any Matrix-compliant clients.\nOur investment in Riot.im represents our commitment to open source projects and the communities that drive them.\nBecause we believe thoughtful, w..."
5065089,"If I was starting out with iOS development today these are the things I would hope somebody would tell me.\nAlthough it's valuable to understand memory management, ARC makes life a whole lot easier.\nEmbrace Open SourceThere are so many amazing libraries and components available for iOS development.\nThere are many examples of this, Tweet Sheets in iOS 5, SKStoreProductViewController & UIActivityViewController in iOS 6, there are many examples.\nHopefully it will be useful to at least a few ..."
9759241,"Next, the gag order.\nReason has published the gag order.\nThe government hasn't unsealed its application for a gag order.\nThat is, it will merely say ""these people said these things, we want their information, therefore, give us a gag order.""\nReason also published AUSA Velamoor's letter asking that the gag order be lifted, and the order lifting it."
13935590,"This article is all about colour switching on the Commodore 64.\nThe co-author of C64 Dragon Breed, Ashley Routledge read the story and took the time to comment with his memories!\nWe're going to generate every unique pair of colours possible on the Commodore 64 in one glorious epilepsy-bomb.\nThe EndAnd that is the story of how I read about secret C64 colours in 1991, and then twenty-six years later finally got to see them.\nCOMMODORE FORMAT issue 5, Dragon Breed review: ""Extra colours are ..."
11521079,"Bill Campbell — who garnered the name “The Coach” for the sage advice and counsel he gave numerous tech leaders from Apple’s Steve Jobs to Google’s Larry Page to Amazon’s Jeff Bezos — has died.\nAnd his family sent this statement: “Bill Campbell passed peacefully in his sleep after a long battle with cancer.\nHe was also an actual football coach at Columbia University way back when, which got him his famous nickname.\n“Steve would say, ‘If you’re helping them you’re hurting me.’ He would yel..."
12476597,"Dynamic Programming: The N

In [ ]:
null
This repository has been archived by the owner.\nIt is now read-only.
You can’t perform that action at this time.\nYou signed in with another tab or window.\nReload to refresh your session.\nYou signed out in another tab or window.\nReload to refresh your session.
Tweet with a locationYou can add location information to your Tweets, such as your city or precise location, from the web and via third-party applications.\nYou always have the option to delete your Tweet location history.
Unfortunately, your browser does not support all of the required features.\nYou can still view a video.
The interactive transcript could not be loaded.\nRating is available when the video has been rented.\nThis feature is not available right now.\nPlease try again later.
Chrome Browser RecommendedThis site may not work correctly in other browsers.\nIf you would like to continue anyway, please click the button below:ContinueOr, check out the downloadable app
This Account has been suspended.\nContact your hosting provider for more information.
The page has moved to: this page
Your report has been saved.\nThanks for your input!
Media playback is unsupported on your device Media caption
Sorry, we cannot display this file.\nSorry, this file is invalid so it cannot be displayed.\nViewer requires iframe

In [156]:
import requests

def fetch_comments(story_id):
    story_json = requests.get('https://hacker-news.firebaseio.com/v0/item/{}.json'.format(story_id)).json()
    comments_ids = story_json["kids"]
    comments = []
    for each_comment_id in comments_ids:
        comment_json = requests.get('https://hacker-news.firebaseio.com/v0/item/{}.json'.format(each_comment_id)).json()
        try:
            commenter = comment_json["by"]
        except:
            commenter = "unknown"
        try:
            comment_text = comment_json["text"]
            comments.append((commenter, comment_text)) 
        except:
            pass
    return comments

In [152]:
comments_df = stories_df[["story_id"]]
comments_df

,story_id
id,
16582136,16582136
11116274,11116274
13682022,13682022
13718752,13718752
11966167,11966167
8532261,8532261
13713480,13713480
15800676,15800676
16319505,16319505


In [157]:
%%time
comments_df["comments"] = series_pmap(comments_df["story_id"], fetch_comments)

JoblibSSLError: JoblibSSLError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Users/jasminetan/anaconda2/envs/py36/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/Users/jasminetan/

In [158]:
comments_df.head()

,story_id
id,
16582136,16582136
11116274,11116274
13682022,13682022
13718752,13718752
11966167,11966167


In [159]:
comments_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25000 entries, 16582136 to 4333774
Data columns (total 1 columns):
story_id    25000 non-null int64
dtypes: int64(1)
memory usage: 1.0 MB
